# Invoking a function or do-it-yourself Agents with Bedrock

## Overview

Conversational interfaces such as chatbots and virtual assistants can be used to enhance the user experience for your customers. These use natural language processing (NLP) and machine learning algorithms to understand and respond to user queries and can be used in a variety of applications, such as customer service, sales, and e-commerce, to provide quick and efficient responses to users. Usually they are augmented by fetching information from various channels such as websites, social media platforms, and messaging apps which involve a complex workflow as shown below


### Chatbot using Amazon Bedrock

![Amazon Bedrock - Agents Interface](../images/agents.jpg)


### Use Cases

1. **QA** - Respond to queries based on look ups
2. **Contextual-aware chatbot** - Dialog turn by turn with external look ups

### Framework for building functions/Agents with Amazon Bedrock
In Conversational interfaces such as chatbots, it is highly important to remember previous interactions, both at a short term but also at a long term level. Further we need to enhance with external tools which are needed to complete the queries. There are many providers which help to create functions or agents. We will look at a vanila implementation for how to do it yourself

### Building  - Key Elements

The first process in a building a contextual-aware chatbot is to identify the tools which can be called by the LLM's. These can be in form of API's invocation or functions calling which can in turn execute code or process data and return the curated results back. 

Second process is the user request orchestration , interaction,  invoking and returning the results.

All of this requires an element of taking action or looking up data dynamically such as invoking an API in the contextual based chatbot. To demonstrate this feature we will build a weather application, which will accept a user input of a place and it will return the weather in real time.

### Architecture [Weather lookup]
We Search and look for the Latitude and Longitude and then invoke the weather app to get predictions

![Amazon Bedrock - Agents Interface](./images/weather.jpg)

---
## Setup `boto3` Connection

In [ ]:
import boto3
import os
from IPython.display import Markdown, display, Pretty

# change to True if running locally and not in sagemaker studio
local = False
if local:
    boto3.setup_default_session(profile_name=os.environ['AWS_PROFILE'])

region = os.environ.get("AWS_REGION")
boto3_bedrock = boto3.client(
    service_name='bedrock-runtime',
    region_name=region,
)

---
## Defining the API Tools

The first thing we need to do for our LLM is define the tools it has access to. In this case we will be defining local Python functions, but it important to not that these could be any type of application service. Examples of what these tools might be on AWS include...

* An AWS Lambda function
* An Amazon RDS database connection
* An Amazon DynamnoDB table
  
More generic examples include...

* REST APIs
* Data warehouses, data lakes, and databases
* Computation engines

In this case, we define two tools which reach external APIs below with two python functions
1. the ability to retrieve the latitude and longitude of a place given a natural language input
2. the ability to retrieve the weather given an input latitude and longitude

In [ ]:
import requests

def get_weather(latitude: str, longitude: str):
    url = f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current_weather=true"
    response = requests.get(url)
    return response.json()

def get_lat_long(place: str):
    url = "https://nominatim.openstreetmap.org/search"
    params = {'q': place, 'format': 'json', 'limit': 1}
    response = requests.get(url, params=params).json()
    if response:
        lat = response[0]["lat"]
        lon = response[0]["lon"]
        return {"latitude": lat, "longitude": lon}
    else:
        return None

def call_function(tool_name, parameters):
    func = globals()[tool_name]
    #print(func, tool_name, parameters)
    output = func(**parameters)
    return output

We also define a function called `call_function` which is used to abstract the tool name. You can see an example of determining the weather in Las Vegas below.

In [ ]:
place = 'Las Vegas'
lat_long_response = call_function('get_lat_long', {'place' : place})
weather_response = call_function('get_weather', lat_long_response)
print(f'Weather in {place} is...')
weather_response

As you might expect, we have to describe our tools to our LLM, so it knows how to use them. The strings below describe the python functions for lat/long and weather to Claude models which can be Claude v-2.0 or Claude Instance v-1.0 in an XML friendly format which we have seen previously in the workshop.

In [ ]:
get_weather_description = """\
<tool_description>
<tool_name>get_weather</tool_name>
<parameters>
<name>latitude</name>
<name>longitude</name>
</parameters>
</tool_description>
"""

get_lat_long_description = """
<tool_description>
<tool_name>get_lat_long</tool_name>
<parameters>
<name>place</name>  
</parameters>
</tool_description>"""

list_of_tools_specs = [get_weather_description, get_lat_long_description]
tools_string = ''.join(list_of_tools_specs)
print(tools_string)

---
## Define Prompts to Orchestrate our LLM Using Tools

Now that the tools are defined both programmatically and as a string, we can start orchestrating the flow which will answer user questions. The first step to this is creating a prompt which defines the rules of operation for Claude. In the prompt below, we provide explicit direction on how Claude should use tools to answer these questions.

In [ ]:
from langchain import PromptTemplate

TOOL_TEMPLATE = """\
Your job is to formulate a solution to a given <user-request> based on the instructions and tools below.

Use these Instructions: 
1. In this environment you have access to a set of tools and functions you can use to answer the question.
2. You can call the functions by using the <function_calls> format below.
3. Only invoke one function at a time and wait for the results before invoking another function.
4. The Results of the function will be in xml tag <function_results>. Never make these up. The values will be provided for you.
5. Only use the information in the <function_results> to answer the question.
6. Once you truly know the answer to the question, place the answer in <answer></answer> tags. Make sure to answer in a full sentence which is friendly.
7. Never ask any questions

<function_calls>
<invoke>
<tool_name>$TOOL_NAME</tool_name>
<parameters>
<$PARAMETER_NAME>$PARAMETER_VALUE</$PARAMETER_NAME>
...
</parameters>
</invoke>
</function_calls>

Here are the tools available:
<tools>
{tools_string}
</tools>

<user-request>
{user_input}
</user-request>

Human: What is the first step in order to solve this problem?

Assistant:
"""
TOOL_PROMPT = PromptTemplate.from_template(TOOL_TEMPLATE)

---
## Executing the RAG Workflow

Armed with our prompt and structured tools, we can now write an orchestration function which will iteratively step through the logical tasks to answer a user question. In the cell below we use the `invoke_model` function to generate a response with Claude and the `single_agent_step` function to iteratively call tools when the LLM tells us we need to. The general flow works like this...

1. The user enters an input to the application
2. The user input is merged with the original prompt and sent to the LLM to determine the next step
3. If the LLM knows the answer, it will answer and we are done. If not, go to next step 4.
4. The LLM will determine which tool to use to answer the question.
5. We will use the tool as directed by the LLM and retrieve the results.
6. We provide the results back into the original prompt as more context.
7. We ask the LLM the next step or if knows the answer.
8. Return to step 3.

This can be a bit over whelming so let us walk through this flow in an example shortly!

In [ ]:
import xmltodict
import json

def invoke_model(prompt):
    client = boto3.client(service_name='bedrock-runtime', region_name=os.environ.get("AWS_REGION"),)
    body = json.dumps({"prompt": prompt, "max_tokens_to_sample": 500, "temperature": 0,})
    modelId = "anthropic.claude-v2"
    #modelId = "anthropic.claude-instant-v1"
    response = client.invoke_model(
        body=body, modelId=modelId, accept="application/json", contentType="application/json"
    )
    return json.loads(response.get("body").read()).get("completion")

def single_agent_step(prompt, output):

    # first check if the model has answered the question
    done = False
    if '<answer>' in output:
        answer = output.split('<answer>')[1]
        answer = answer.split('</answer>')[0]
        done = True
        return done, answer
    
    # if the model has not answered the question, go execute a function
    else:

        # parse the output for any 
        function_xml = output.split('<function_calls>')[1]
        function_xml = function_xml.split('</function_calls>')[0]
        function_dict = xmltodict.parse(function_xml)
        func_name = function_dict['invoke']['tool_name']
        parameters = function_dict['invoke']['parameters']

        #print(f"single_agent_step:: func_name={func_name}::params={parameters}::function_dict={function_dict}::")
        # call the function which was parsed
        func_response = call_function(func_name, parameters)

        # create the next human input
        func_response_str = '\n\nHuman: Here is the result from your function call\n\n'
        func_response_str = func_response_str + f'<function_results>\n{func_response}\n</function_results>'
        func_response_str = func_response_str + '\n\nIf you know the answer, say it. If not, what is the next step?\n\nAssistant:'

        # augment the prompt
        prompt = prompt + output + func_response_str
    return done, prompt

Let's start our first example `What is the weather in Las Vegas?`. The code below asks the LLM what the first step is and you will notice that the LLM is able to ascertain it needs to use the `get_lat_long` tool first.

In [ ]:
user_input = 'What is the weather in Las Vegas?'
next_step = TOOL_PROMPT.format(tools_string=tools_string, user_input=user_input)

output = invoke_model(next_step).strip()
done, next_step = single_agent_step(next_step, output)
if not done:
    display(Pretty(f'{output}'))
else:
    display(Pretty('Final answer from LLM:\n'+f'{next_step}'))

Great, Claude has figured out that we should first call the lat and long tool. The next step is then orchestrated just like the first. This time, Claude uses the lat/long from the first request to now ask for the weather of that specific location.

In [ ]:
output = invoke_model(next_step).strip()
done, next_step = single_agent_step(next_step, output)
if not done:
    display(Pretty(f'{output}'))
else:
    display(Pretty('Final answer from LLM:\n'+f'{next_step}'))

Finally the LLM is able to answer the question based on the input function above. Very cool!

In [ ]:
output = invoke_model(next_step).strip()
done, next_step = single_agent_step(next_step, output)
if not done:
    display(Pretty(f'{output}'))
else:
    display(Pretty('Final answer from LLM:\n'+f'{next_step}'))

Let's try another example to show how a different place (Singapore) can be used in this example. Notice how we set the for loop to 5 iterations even though the model only uses 3 of these. This iteration capping is common in agent workflows and should be tuned according to your use case. 

In [ ]:
user_input = 'What is the weather in Singapore?'
next_step = TOOL_PROMPT.format(tools_string=tools_string, user_input=user_input)

for i in range(5):
    output = invoke_model(next_step).strip()
    done, next_step = single_agent_step(next_step, output)
    if not done:
        display(Pretty(f'{output}'))
    else:
        display(Pretty('Final answer from LLM:\n'+f'{next_step}'))
        break

---
## Leveraging Open Source Agents with Bedrock

As with the previous notebook, there are many ways to orchestrate agent workflows with LLMs. One of the popular open source frameworks for this is [LangChain's agent module](https://python.langchain.com/docs/modules/agents/). LangChain provides a variety of agent types as well as the same [integration to Amazon Bedrock](https://python.langchain.com/docs/integrations/llms/bedrock) being available for these agent based workflows. Let's work on implementing the same workflow as above into the LangChain ecosystem.

First, we can define our LLM with the `Bedrock` class.

In [ ]:
from langchain.llms.bedrock import Bedrock

agent_llm = Bedrock(
    client=boto3.client(service_name='bedrock-runtime', region_name=os.environ.get("AWS_REGION"),),
    model_id="anthropic.claude-instant-v1",
    model_kwargs={
        "max_tokens_to_sample": 500,
        "temperature": 0.0,
    },
)

Now we will use the same python code as our original implementation, but we will use the `@tool` decorator to specify that this python function is able to be used as tool for LLMs.

In [ ]:
from langchain.tools import tool
import ast

@tool ("get_lat_long")
def get_lat_long(place: str) -> dict:
    """Returns the latitude and longitude for a given place name as a dict object of python."""
    url = "https://nominatim.openstreetmap.org/search"

    params = {'q': place, 'format': 'json', 'limit': 1}
    response = requests.get(url, params=params).json()

    if response:
        lat = response[0]["lat"]
        lon = response[0]["lon"]
        return {"latitude": lat, "longitude": lon}
    else:
        return None

@tool ("get_weather_dict")
def get_weather_dict(co_ord_str: str) -> dict:
    """
    Returns weather data for a given a dictionary having latitude and longitude.
    """
    if '{' in co_ord_str:
        co_ord_str = ast.literal_eval(co_ord_str)
        latitude = co_ord_str['latitude']
        longitude = co_ord_str['longitude']
    else:
        latitude = float(co_ord_str.split(",")[0].strip())
        longitude = float(co_ord_str.split(",")[1].strip())
    url = f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current_weather=true"
    response = requests.get(url)
    return response.json()

As well as being able to define our own tools, LangChain has some pre-built tools which are general purpose. One well documented issue with LLMs is that they are not good at arithmetic only from natural language. We can use the `LLMMathChain` from LangChain to help solve this problem. In this chain, we are able to ask an LLM simple math questions and the question will be sent to a calculator in order to return a perfect mathematical result. An example is included below.

In [ ]:
from langchain import LLMMathChain

math_tool_template = """\
Given a question from the Human with a math problem, provide only a single line mathematical expression that solves the problem in the following format.
Never solve the expression only create a parsable expression.
```text
${{single line mathematical expression that solves the problem}}
```

Here is an example response with a single line mathematical expression for solving a math problem:
```text
37593**(1/5)
```

Human: {question}

Assistant:"""

llm_math_chain = LLMMathChain.from_llm(llm=agent_llm, verbose=True)
llm_math_chain.llm_chain.prompt.template = math_tool_template
result = llm_math_chain('What is nine times 465?')

Now that all the tools are defined, lets create a [ReACT agent](https://python.langchain.com/docs/modules/agents/agent_types/react) (Reason + Act) which will leverage the LLM to generate the next set of actions to solve a given question. This next cell creates the agent from a list of tools we provide.

In [ ]:
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

tools_list = [get_lat_long, get_weather_dict]
tools_list.append(
    Tool.from_function(
        func=llm_math_chain.run,
        name="Calculator",
        description="Useful for when you need to answer questions about math.",
    )
)
react_agent = initialize_agent(
    tools_list, 
    agent_llm, 
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
    verbose=True,
    max_iteration=4,
    return_intermediate_steps=True,
    handle_parsing_errors=True,
)

The ReAct framework also expects a specific format to be followed by our LLM. This custom prompt adds place holders for the 3 components which need to be followed.

1. **Action** which is the tool to use
2. **Action Input** is the parameters coming from the LLM which will go into the tool or function to be called
3. **Observation** is the final result of the call from the tool

These three components to ReAct are recursively executed until there is an answer to the question or we reach the limit of execution steps.

In [ ]:
prompt_template = """Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do, Also try to follow steps mentioned above
Action: the action to take, should be one of [ "get_lat_long", "get_weather_dict", "Calculator"]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Question: {input}

Assistant:
{agent_scratchpad}"""

react_agent.agent.llm_chain.prompt.template=prompt_template

Now we can send in the first question just like the one we used in the DIY system. Notice how the logging allows you to see how the LLM is reasoning through the task within the ReAct framework.

In [ ]:
result = react_agent("Can you check the weather in Las Vegas for me?")

Another example is shown below.

In [ ]:
result = react_agent("can you check the weather in Seattle WA for me?")

In this last example, we will give a different type of problem to the LLM where it will be able to take advantage of the math tool as well as the lat/long tool instead of only talking about weather.

In [ ]:
result = react_agent("what is the latitude of Washington DC multiplied by 2")

---
## Next steps

Now that we have a working RAG application with vector search retrieval, we will explore a new type of retrieval. In the next notebook we will see how to use LLM agents to automatically retrieve information from APIs.